In [18]:
from mutation.controller import run_trials

from mutation import controller
from mutation import maker
from mutation import transformers

from pathlib import Path
import ast

In [65]:
pkg_dir = Path("firstmodule")

In [66]:
controller.clean_trial(pkg_dir)

2018-12-26 10:30:19,343 - mutation.cache - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/__init__.cpython-37.pyc
2018-12-26 10:30:19,344 - mutation.cache - DEBUG - Removing cache file: /home/zassoc/Github/m/firstmodule/__pycache__/run.cpython-37.pyc
2018-12-26 10:30:19,345 - mutation.controller - DEBUG - Running clean trial


In [67]:
src_trees, src_targets = controller.build_src_trees_and_targets(pkg_dir)
sample_space = controller.get_sample_space(src_targets)

2018-12-26 10:30:19,827 - mutation.controller - INFO - Creating AST from: /home/zassoc/Github/m/firstmodule/__init__.py
2018-12-26 10:30:19,829 - mutation.controller - INFO - Get mutation targets from AST.
2018-12-26 10:30:19,831 - mutation.controller - INFO - Creating AST from: /home/zassoc/Github/m/firstmodule/run.py
2018-12-26 10:30:19,834 - mutation.controller - INFO - Get mutation targets from AST.
2018-12-26 10:30:19,835 - mutation.transformers - INFO - NODE
2018-12-26 10:30:19,837 - mutation.transformers - INFO - BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=10))
2018-12-26 10:30:19,838 - mutation.transformers - INFO - IDX VALUE:
2018-12-26 10:30:19,840 - mutation.transformers - INFO - LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>)
2018-12-26 10:30:19,841 - mutation.transformers - INFO - NODE
2018-12-26 10:30:19,842 - mutation.transformers - INFO - BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=5))
2018-12-26 10:30:19,84

In [68]:
src_trees

{'/home/zassoc/Github/m/firstmodule/run.py': <_ast.Module at 0x7f03c9684400>}

In [69]:
src_targets

{'/home/zassoc/Github/m/firstmodule/run.py': [LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>),
  LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>)]}

In [70]:
sample_space

[('/home/zassoc/Github/m/firstmodule/run.py',
  LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>)),
 ('/home/zassoc/Github/m/firstmodule/run.py',
  LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>))]

In [71]:
src, tgt = sample_space[0]

In [72]:
src

'/home/zassoc/Github/m/firstmodule/run.py'

In [73]:
tgt

LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>)

In [74]:
src_tree = src_trees[src]

In [75]:
ast.dump(src_tree)

"Module(body=[FunctionDef(name='myfunc', args=arguments(args=[arg(arg='a', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Str(s='hello'), Name(id='a', ctx=Load())], keywords=[]))], decorator_list=[], returns=None), FunctionDef(name='add_ten', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=10)))], decorator_list=[], returns=None), FunctionDef(name='add_five', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=5)))], decorator_list=[], returns=None), Expr(value=Call(func=Name(id='myfunc', ctx=Load()), args=[Str(s='there')], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()

In [76]:
mutant_ops = transformers.get_mutations_for_target(tgt)

2018-12-26 10:30:33,108 - mutation.transformers - DEBUG - Potential mutation operations found for target: <class '_ast.Add'>


In [77]:
mutant_ops

{_ast.Div, _ast.Mult, _ast.Sub}

In [61]:
from typing import NamedTuple

class LocIndex(NamedTuple):
    ast_class: str
    lineno: int
    col_offset:int
    op_type: type

class MutateAST(ast.NodeTransformer):

    def __init__(self, readonly=False, target_idx=None, mutation=None):
        self.readonly = readonly
        self.locs = set()
        self.target_idx = target_idx
        self.mutation = mutation


    def visit_BinOp(self, node):

        self.generic_visit(node)

        idx = LocIndex("BinOp", node.lineno, node.col_offset, type(node.op))
        self.locs.add(idx)

        if self.readonly:
            print("READ ONLY NODE")
            return node

        else:
            print("ELSE STATEMENT")
            if idx == self.target_idx and self.mutation:
                print("ELSE MUTATION CONDITION")
                print(self.mutation, "\n\n")
                return ast.copy_location(
                    #ast.BinOp(left=node.left, op=ast.Sub(), right=node.right),
                    ast.BinOp(left=node.left, op=self.mutation(), right=node.right),
                    node)
            else:
                print("ELSE RETURN NODE")
                return node

In [78]:
from copy import deepcopy

mutant_ops = transformers.get_mutations_for_target(tgt)

while mutant_ops:
    mutation_tree = deepcopy(src_tree)
    op = mutant_ops.pop()
    print(op, "\n")
    print(mutation_tree == src_tree)
    tree2 = MutateAST(readonly=False, target_idx=tgt, mutation=op).visit(mutation_tree)
    print(ast.dump(tree2) + "\n\n\n")

2018-12-26 10:31:24,005 - mutation.transformers - DEBUG - Potential mutation operations found for target: <class '_ast.Add'>
<class '_ast.Sub'> 

False
ELSE STATEMENT
ELSE MUTATION CONDITION
<class '_ast.Sub'> 


ELSE STATEMENT
ELSE RETURN NODE
Module(body=[FunctionDef(name='myfunc', args=arguments(args=[arg(arg='a', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Str(s='hello'), Name(id='a', ctx=Load())], keywords=[]))], decorator_list=[], returns=None), FunctionDef(name='add_ten', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Sub(), right=Num(n=10)))], decorator_list=[], returns=None), FunctionDef(name='add_five', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(

In [79]:
tree2.body

In [51]:
ast.dump(tree2)

"Module(body=[FunctionDef(name='myfunc', args=arguments(args=[arg(arg='a', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Str(s='hello'), Name(id='a', ctx=Load())], keywords=[]))], decorator_list=[], returns=None), FunctionDef(name='add_ten', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Sub(), right=Num(n=10)))], decorator_list=[], returns=None), FunctionDef(name='add_five', args=arguments(args=[arg(arg='b', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Name(id='b', ctx=Load()), op=Add(), right=Num(n=5)))], decorator_list=[], returns=None), Expr(value=Call(func=Name(id='myfunc', ctx=Load()), args=[Str(s='there')], keywords=[])), Expr(value=Call(func=Name(id='print', ctx=Load()